<a href="https://colab.research.google.com/github/williamvpaiva/PowerBI-DataScience/blob/master/VERS%C3%83O_FINAL_Analise_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3MB 4.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=f4b1febedd8838be3c17ce712316a7b5a84afcdebe9c92c5c46de80d01b5659f
  Stored in directory: /tmp/pip-ephem-wheel-cache-dsuancwy/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly
import openpyxl

In [ ]:
%%time

# Imports
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

# Definindo características da página
url = 'https://www.fundamentus.com.br/resultado.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

# Fazendo a request
try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    print('Foi\n\n')
    print(response.getcode())
    html = response.read()

# Tratando alguns possíveis erros
except HTTPError as e:
    print('HTTPError\n\n')
    print(response.getcode())
    print(e.status, e.reason)

except URLError as e:
    print('URLError\n\n')
    print(response.getcode())
    print(e.reason)

# Instanciando um objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Pegando os nomes das colunas da tabela
colunas_names = [col.getText() for col in soup.find('table', {'id': 'resultado'}).find('thead').findAll('th')]
colunas = {i: col.getText() for i, col in enumerate(soup.find('table', {'id': 'resultado'}).find('thead').findAll('th'))}

# Criando um DataFrame com os nomes das colunas
dados = pd.DataFrame(columns=colunas_names)

# Pegando os dados da tabela por linha
for i in range(len(soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr'))):
    linha = soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr')[i].getText().split('\n')[1:]
    inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
    dados = pd.concat([dados, inserir_linha], ignore_index=True)

# É tetra!
dados

Foi


200
CPU times: user 35.3 s, sys: 127 ms, total: 35.4 s
Wall time: 35.8 s


#Renomeando colunas

In [ ]:
df = dados.rename(columns={'Div.Yield':'DY',
                        'Mrg Ebit':'MrgEbit',
                        'Mrg. Líq.':'MrgLíq',
                        'Liq. Corr.':'LiqCorr',
                        'Patrim. Líq':'PatrimLíq',
                        'Dív.Brut/ Patrim.':'DívBrutPatrim',
                        'Cresc. Rec.5a':'CrescRec5a',
                        'Liq.2meses':'Liq2meses'
                        #'DY (12M)Acumulado':'DY12MA',
                       # 'DY (3M)Média':'DY(3M)Media',
                        #'DY (6M)Média':'DY(6M)Media',
                       # 'DY Ano':'DYAno',
                        #'PatrimônioLíq.':'Patrimonio'
                        #'QuantidadeAtivos': 'ativos'
#, 'PatrimônioLíq.':'Patr Liqu'
})
df

,Papel,Cotação,P/L,P/VP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,MrgEbit,MrgLíq,LiqCorr,ROIC,ROE,Liq2meses,PatrimLíq,DívBrutPatrim,CrescRec5a,21
0,MNSA3,"0,42","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","-208,15%","-362,66%","3,63","-13,50%","145,70%","0,00","-9.105.000,00","-6,52","-41,11%",
1,CLAN3,"0,00","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","-1,05%","0,00","1.012.240.000,00","0,00","-63,96%",
2,CSTB3,"150,00","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","40,85%","28,98%","2,60","22,40%","20,11%","0,00","8.420.670.000,00","0,14","31,91%",
3,POPR4,"10,17","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","8,66%","5,65%","1,08","15,25%","19,93%","0,00","545.803.000,00","0,82","30,93%",
4,MNSA4,"0,47","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","-208,15%","-362,66%","3,63","-13,50%","145,70%","0,00","-9.105.000,00","-6,52","-41,11%",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,BIDI4,"59,10","2.574,87","13,94","0,000","0,08%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","173.840.000,00","3.240.140.000,00","0,00","30,91%",
950,BIDI11,"178,87","2.597,67","14,07","0,000","0,08%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","306.760.000,00","3.240.140.000,00","0,00","30,91%",
951,BIDI3,"59,99","2.613,65","14,15","0,000","0,07%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","20.264.100,00","3.240.140.000,00","0,00","30,91%",
952,NTCO3,"50,37","4.493,39","2,42","1,784","0,00%","1,122","25,37","28,22","-4,51","31,25","14,63","6,32%","0,01%","1,18","5,05%","0,05%","294.332.000,00","28.691.300.000,00","0,49","0,00%",


# Tratando as colunas e convertendo em float

In [ ]:

df['Cotação'] = df['Cotação'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/L'] = df['P/L'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/VP'] = df['P/VP'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PSR'] = df['PSR'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['DY'] = df['DY'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Ativo'] = df['P/Ativo'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Cap.Giro'] = df['P/Cap.Giro'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/EBIT'] = df['P/EBIT'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Ativ Circ.Liq'] = df['P/Ativ Circ.Liq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['EV/EBIT'] = df['EV/EBIT'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['EV/EBITDA'] = df['EV/EBITDA'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['MrgEbit'] = df['MrgEbit'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['MrgLíq'] = df['MrgLíq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['LiqCorr'] = df['LiqCorr'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['ROIC'] = df['ROIC'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['ROE'] = df['ROE'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['Liq2meses'] = df['Liq2meses'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PatrimLíq'] = df['PatrimLíq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['DívBrutPatrim'] = df['DívBrutPatrim'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['CrescRec5a'] = df['CrescRec5a'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))	

#Filtros

In [ ]:
# Crescimento superior a 10% nos ultimos 5 anos
df = df.query('CrescRec5a >= 10.00')  

In [ ]:
# Liquidez ultimos 2 meses maior que ou igual a 200 mil
df = df.query('Liq2meses	 >= 200000')  

In [ ]:
# Media de DY (12M)Acumulado maior ou ugual a 0,40%
df = df.query('PatrimLíq >= 1500000000')

In [ ]:
# DY (12M) Média maior ou ugual a 0,40%
df = df.query('ROE >= 15.00')

In [ ]:
# DY mensal maior ou igual a 0.40%
df =  df.query('LiqCorr >= 1.50')

In [ ]:
df

,Papel,Cotação,P/L,P/VP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,MrgEbit,MrgLíq,LiqCorr,ROIC,ROE,Liq2meses,PatrimLíq,DívBrutPatrim,CrescRec5a,21
447,CYRE3,21.95,4.56,1.54,2.160,13.23,0.753,2.23,14.26,17.76,15.54,14.61,15.15,48.97,2.78,6.43,33.79,1.343200e+08,5.696170e+09,0.47,10.74,
455,TASA4,24.15,4.95,15.77,1.194,0.00,1.425,5.86,4.37,-5.59,5.37,5.11,27.35,24.13,1.59,43.80,318.75,5.564100e+07,1.533560e+08,5.29,21.94,
456,TASA3,24.28,4.98,15.86,1.200,0.00,1.432,5.89,4.39,-5.62,5.39,5.13,27.35,24.13,1.59,43.80,318.75,2.583240e+06,1.533560e+08,5.29,21.94,
464,RAPT3,12.64,5.49,2.00,0.716,4.81,0.460,1.59,5.66,-4.68,7.85,6.29,12.64,16.47,1.93,12.41,36.45,2.817930e+05,2.183370e+09,1.81,25.34,
465,TAEE3,13.10,5.52,2.05,3.582,13.53,0.901,7.86,5.16,-2.33,7.23,7.19,69.43,64.95,2.93,18.85,37.20,1.468430e+06,6.598210e+09,0.98,32.43,
468,TAEE4,13.16,5.54,2.06,3.599,13.46,0.905,7.89,5.18,-2.34,7.25,7.21,69.43,64.95,2.93,18.85,37.20,3.541220e+06,6.598210e+09,0.98,32.43,
469,TAEE11,39.52,5.55,2.06,3.602,13.45,0.906,7.90,5.19,-2.35,7.26,7.21,69.43,64.95,2.93,18.85,37.20,9.052350e+07,6.598210e+09,0.98,32.43,
474,MTSA4,41.67,5.67,1.32,0.937,4.47,0.920,1.83,7.68,1.95,6.55,5.58,12.20,16.54,2.83,17.17,23.32,2.128600e+05,2.881250e+08,0.18,15.03,
484,CSNA3,45.70,6.10,4.41,1.730,1.43,0.910,4.79,4.95,-2.80,6.20,5.17,34.93,30.84,1.80,28.24,72.25,5.698780e+08,1.438750e+10,2.37,18.64,
487,JHSF3,7.35,6.19,1.29,3.646,4.77,0.665,4.34,6.12,-3.20,6.69,6.42,59.54,58.42,2.35,13.01,20.79,4.055380e+07,3.933220e+09,0.42,40.08,


In [ ]:
#df.to_excel('fundos_imobiliarios_.xlsx')
df.to_excel('acoes.xlsx')

In [ ]:
#df = pd.read_csv('/content/fundos_imobiliarios_.csv', sep=';', encoding='latin-1',decimal='.', thousands='.')
df = pd.read_excel('/content/acoes.xlsx')
df

,Unnamed: 0,Papel,Cotação,P/L,P/VP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,MrgEbit,MrgLíq,LiqCorr,ROIC,ROE,Liq.2meses,PatrimLíq,DívBrutPatrim,CrescRec5a,21
0,0,MNSA4,"0,47","0,00","0,00","0,000",0.00,"0,000","0,00","0,00","0,00","0,00","0,00",-208.15,-362.66,3.63,-13.50,145.70,"0,00","-9.105.000,00","-6,52",-41.11,NaN
1,1,POPR4,"10,17","0,00","0,00","0,000",0.00,"0,000","0,00","0,00","0,00","0,00","0,00",8.66,5.65,1.08,15.25,19.93,"0,00","545.803.000,00","0,82",30.93,NaN
2,2,PMET3,"0,00","0,00","0,00","0,000",0.00,"0,000","0,00","0,00","0,00","0,00","0,00",0.00,0.00,0.00,0.00,4.10,"0,00","-290.863.000,00","0,00",37.74,NaN
3,3,CSTB4,"147,69","0,00","0,00","0,000",0.00,"0,000","0,00","0,00","0,00","0,00","0,00",40.85,28.98,2.60,22.40,20.11,"0,00","8.420.670.000,00","0,14",31.91,NaN
4,4,CFLU4,"1.000,00","0,00","0,00","0,000",0.00,"0,000","0,00","0,00","0,00","0,00","0,00",8.88,10.72,1.10,17.68,32.15,"0,00","60.351.000,00","0,06",8.14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,949,BIDI4,"59,10","2.574,87","13,94","0,000",0.08,"0,000","0,00","0,00","0,00","0,00","0,00",0.00,0.00,0.00,0.00,0.54,"173.840.000,00","3.240.140.000,00","0,00",30.91,NaN
950,950,BIDI11,"178,87","2.597,67","14,07","0,000",0.08,"0,000","0,00","0,00","0,00","0,00","0,00",0.00,0.00,0.00,0.00,0.54,"306.760.000,00","3.240.140.000,00","0,00",30.91,NaN
951,951,BIDI3,"59,99","2.613,65","14,15","0,000",0.07,"0,000","0,00","0,00","0,00","0,00","0,00",0.00,0.00,0.00,0.00,0.54,"20.264.100,00","3.240.140.000,00","0,00",30.91,NaN
952,952,NTCO3,"50,37","4.493,39","2,42","1,784",0.00,"1,122","25,37","28,22","-4,51","31,25","14,63",6.32,0.01,1.18,5.05,0.05,"294.332.000,00","28.691.300.000,00","0,49",0.00,NaN


In [ ]:
df.dtypes

Unnamed: 0           int64
Papel               object
Cotação             object
P/L                 object
P/VP                object
PSR                 object
DY                  object
P/Ativo             object
P/Cap.Giro          object
P/EBIT              object
P/Ativ Circ.Liq     object
EV/EBIT             object
EV/EBITDA           object
MrgEbit             object
MrgLíq              object
LiqCorr            float64
ROIC                object
ROE                 object
Liq.2meses          object
PatrimLíq           object
DívBrutPatrim       object
CrescRec5a          object
21                 float64
dtype: object

In [ ]:
symbols = df['Papel']+'.SA'
symbols

0       MNSA4.SA
1       POPR4.SA
2       PMET3.SA
3       CSTB4.SA
4       CFLU4.SA
         ...    
949     BIDI4.SA
950    BIDI11.SA
951     BIDI3.SA
952     NTCO3.SA
953    BBTG12.SA
Name: Papel, Length: 954, dtype: object

In [ ]:
start = datetime(2019, 1, 1)
end = datetime.today()

In [ ]:
def pega_fundos(name):
  stock = yf.download(name, start = start, end = end)
  return stock

In [ ]:
stock_dict = {}

for symbol in symbols:
  try:
    stock_dict[symbol] = pega_fundos(symbol)
  except:
    print('NÃO DEU CERTO PARA O TICKET',symbol)
print('TERMINOU')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- MNSA4.SA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- POPR4.SA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PMET3.SA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CSTB4.SA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CFLU4.SA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- IVTT3.SA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed downloa

In [ ]:
stock_dict['ITSA4.SA']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,12.11,12.70,12.08,12.65,11.247989,25162700
2019-01-03,12.65,12.76,12.51,12.76,11.345797,37892900
2019-01-04,12.66,12.83,12.56,12.72,11.310229,32715200
2019-01-07,12.70,12.82,12.62,12.71,11.301338,13436300
2019-01-08,12.71,12.88,12.55,12.88,11.452496,17298500
...,...,...,...,...,...,...
2021-05-17,10.61,10.65,10.45,10.65,10.650000,28019000
2021-05-18,10.55,10.71,10.55,10.62,10.620000,18899500
2021-05-19,10.54,10.74,10.49,10.74,10.740000,21080600


In [ ]:
returns = pd.DataFrame()

for key in stock_dict.keys():
  returns[key] = stock_dict[key]['Adj Close'].pct_change()

returns.head()

,LVBI11.SA,KNHY11.SA,JSRE11.SA,XPLG11.SA,HGLG11.SA,HGRE11.SA,TGAR11.SA,BBPO11.SA,VRTA11.SA,HFOF11.SA,MALL11.SA,KNIP11.SA,KNRI11.SA,MXRF11.SA,HABT11.SA,HGRU11.SA,XPPR11.SA,SDIL11.SA,XPCI11.SA,HGCR11.SA,BRCO11.SA,VINO11.SA,VILG11.SA,GTWR11.SA,TRXF11.SA,GGRC11.SA,BRCR11.SA,BCFF11.SA,HCTR11.SA,RBRL11.SA,MGFF11.SA,RECT11.SA,RECR11.SA,CPTS11.SA,CVBI11.SA,IRDM11.SA,RBRF11.SA,RBRR11.SA,MCCI11.SA,VGIP11.SA,BTLG11.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-24,NaN,-0.026035,0.049383,NaN,0.049554,0.073740,0.0,0.023226,0.003332,0.029240,0.056975,-0.033066,0.049057,0.050410,0.027215,0.020200,0.054026,0.070423,0.007326,0.025285,0.055437,0.031191,0.034091,0.000200,0.0,0.020619,0.050904,0.0,0.010319,NaN,0.002703,0.011780,NaN,NaN,-0.020610,NaN,0.078112,0.037252,0.018301,0.047619,0.037565
2020-03-25,0.090868,0.027284,0.058824,NaN,0.089357,0.101106,0.0,0.045082,0.021538,0.042159,0.108108,0.034197,0.071223,0.029018,0.046211,0.039012,0.104320,0.111842,0.101515,0.023439,0.010505,0.154468,0.131868,0.011821,0.0,0.061818,0.064941,0.0,0.012687,NaN,0.051078,0.056695,NaN,NaN,0.027327,NaN,0.132075,0.077538,0.154814,0.157424,0.064420
2020-03-26,0.046463,0.096774,-0.011111,NaN,0.041833,0.083853,0.0,0.014253,0.043350,0.012976,0.084011,0.032064,0.019745,0.052061,0.030742,0.014151,0.096855,0.088402,0.053645,0.022468,0.009596,0.059800,0.067961,0.007426,0.0,0.046423,-0.007804,0.0,0.077609,NaN,0.064239,0.047784,NaN,NaN,0.109388,NaN,0.022222,0.068190,-0.021787,0.007985,0.020291
2020-03-27,0.029600,0.021863,0.006742,NaN,0.012587,0.029348,0.0,-0.003643,-0.013220,0.009150,0.062500,0.004369,-0.008166,0.023711,0.011313,0.022326,-0.013761,0.083940,0.030548,-0.006369,0.011089,0.034157,0.027182,0.056020,0.0,0.023636,0.008315,0.0,-0.014680,NaN,0.021084,0.017259,NaN,NaN,0.000000,NaN,0.000000,-0.019846,-0.025000,0.000000,-0.006782
2020-03-30,-0.014180,0.007387,0.038170,NaN,-0.011747,-0.057643,0.0,0.031343,-0.004976,0.022400,-0.043765,0.020880,0.016268,0.026183,0.007119,0.004550,-0.028953,-0.012037,0.012161,0.025641,0.067372,-0.003650,0.002213,-0.027455,0.0,0.038632,0.000780,0.0,0.008899,NaN,-0.014749,0.011236,NaN,NaN,0.000000,NaN,-0.021739,0.001125,0.017483,0.010390,0.008102


In [ ]:
returns.describe()

,LVBI11.SA,KNHY11.SA,JSRE11.SA,XPLG11.SA,HGLG11.SA,HGRE11.SA,TGAR11.SA,BBPO11.SA,VRTA11.SA,HFOF11.SA,MALL11.SA,KNIP11.SA,KNRI11.SA,MXRF11.SA,HABT11.SA,HGRU11.SA,XPPR11.SA,SDIL11.SA,XPCI11.SA,HGCR11.SA,BRCO11.SA,VINO11.SA,VILG11.SA,GTWR11.SA,TRXF11.SA,GGRC11.SA,BRCR11.SA,BCFF11.SA,HCTR11.SA,RBRL11.SA,MGFF11.SA,RECT11.SA,RECR11.SA,CPTS11.SA,CVBI11.SA,IRDM11.SA,RBRF11.SA,RBRR11.SA,MCCI11.SA,VGIP11.SA,BTLG11.SA
count,285.000000,286.000000,286.000000,236.000000,286.000000,286.000000,48.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,48.000000,286.000000,0.0,286.000000,286.000000,0.0,0.0,286.000000,236.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,0.000836,0.000656,0.000428,-0.000069,0.000929,0.000295,0.002575,-0.000204,0.000428,0.000537,0.001640,0.000328,0.000365,0.000764,0.001618,0.000582,-0.000080,0.001130,0.001525,0.000673,0.000560,0.001360,0.001109,0.000256,0.000495,0.000806,0.000337,0.001674,0.001503,NaN,0.000517,0.000098,NaN,NaN,0.001506,0.000720,0.000805,0.001048,0.001101,0.001938,0.001312
std,0.015463,0.010025,0.011450,0.009868,0.013680,0.014444,0.017837,0.011616,0.009433,0.006962,0.014193,0.008272,0.009555,0.009405,0.009537,0.009658,0.017889,0.014641,0.012620,0.008820,0.011691,0.013909,0.013859,0.009674,0.015086,0.011591,0.009032,0.011597,0.013360,NaN,0.011531,0.010614,NaN,NaN,0.012927,0.013659,0.016217,0.011275,0.014159,0.019156,0.010095
min,-0.049955,-0.036697,-0.038650,-0.042148,-0.090442,-0.057643,0.000000,-0.070312,-0.039538,-0.025126,-0.043765,-0.033066,-0.027273,-0.052415,-0.052888,-0.041707,-0.068462,-0.043143,-0.057617,-0.041028,-0.035462,-0.042346,-0.072596,-0.048331,-0.151042,-0.057123,-0.049495,0.000000,-0.045736,NaN,-0.042162,-0.033599,NaN,NaN,-0.045995,-0.096513,-0.058605,-0.025696,-0.035438,-0.123258,-0.059146
25%,-0.006790,-0.003308,-0.005639,-0.004313,-0.004255,-0.006056,0.000000,-0.003099,-0.002927,-0.002009,-0.003688,-0.003963,-0.004645,-0.002734,-0.002110,-0.003652,-0.006276,-0.004269,-0.004007,-0.003171,-0.005201,-0.003967,-0.003142,-0.003785,-0.002120,-0.005313,-0.003788,0.000000,-0.004159,NaN,-0.004366,-0.003063,NaN,NaN,-0.004465,-0.003649,-0.004666,-0.004023,-0.004046,-0.003836,-0.002693
50%,0.000000,0.000261,0.000000,0.000242,0.000085,-0.000099,0.000000,0.000143,0.000661,0.000000,-0.000052,0.000306,-0.000180,0.000456,0.001073,0.000000,0.000116,-0.000203,0.001107,0.000101,0.000000,0.000322,0.000254,0.000085,0.000000,0.000739,0.000000,0.000000,0.001366,NaN,0.000000,0.000102,NaN,NaN,0.000746,0.001777,0.000051,0.000099,0.000000,0.000303,0.000646
75%,0.006909,0.004596,0.005324,0.003817,0.005524,0.004502,0.000000,0.003660,0.004088,0.002134,0.005174,0.004421,0.003563,0.003788,0.005067,0.004419,0.006548,0.003757,0.006200,0.004837,0.004788,0.004795,0.004510,0.004368,0.002392,0.006569,0.003474,0.000000,0.007809,NaN,0.003884,0.003638,NaN,NaN,0.006257,0.007518,0.004517,0.005135,0.004746,0.006087,0.005014
max,0.090868,0.096774,0.062932,0.049838,0.089357,0.101106,0.123577,0.048749,0.043350,0.042159,0.108108,0.034197,0.071223,0.052061,0.049127,0.039012,0.104320,0.111842,0.101515,0.031418,0.067372,0.154468,0.131868,0.056020,0.177301,0.061818,0.064941,0.080345,0.077609,NaN,0.064239,0.056695,NaN,NaN,0.109388,0.030248,0.132075,0.077538,0.154814,0.157424,0.064420


In [ ]:
discount_dict = {}
def calcula_desconto(df):
  max_price = df['Adj Close'].describe()['75%']
  last_price = df['Adj Close'].iloc[-1]
  return ((last_price - max_price)/max_price)*100

In [ ]:
for tick in stock_dict.keys():
  try:
    discount_dict[tick] =  calcula_desconto(stock_dict[tick])
  except:
    discount_dict[tick] = 0
discount_dict

{'BBPO11.SA': -23.589447355451078,
 'BCFF11.SA': 8.034464635229682,
 'BRCO11.SA': -13.062725628013677,
 'BRCR11.SA': -10.789514552076252,
 'BTLG11.SA': 1.8441066092907183,
 'CPTS11.SA': 0.0,
 'CVBI11.SA': -0.04285109340395933,
 'GGRC11.SA': -15.465737767540071,
 'GTWR11.SA': -8.3859229720697,
 'HABT11.SA': 4.696681660122685,
 'HCTR11.SA': -4.6216056666864365,
 'HFOF11.SA': -8.841878257477987,
 'HGCR11.SA': -10.987081321028988,
 'HGLG11.SA': -5.01660687029856,
 'HGRE11.SA': -19.150067988726807,
 'HGRU11.SA': -9.618888172411957,
 'IRDM11.SA': -2.4727228338068183,
 'JSRE11.SA': -18.457521049843475,
 'KNHY11.SA': -4.262554361478999,
 'KNIP11.SA': -6.017313375101461,
 'KNRI11.SA': -12.53275743863664,
 'LVBI11.SA': -12.24868590208017,
 'MALL11.SA': -8.317529477169211,
 'MCCI11.SA': -0.9806724223568896,
 'MGFF11.SA': -15.378428966668734,
 'MXRF11.SA': -7.338645858410146,
 'RBRF11.SA': -14.729419987923517,
 'RBRL11.SA': 0.0,
 'RBRR11.SA': -4.816370386496249,
 'RECR11.SA': 0.0,
 'RECT11.SA': -2

In [ ]:
def pega_setor(stock):
  return[x for x  in df[symbols == stock]['Setor']]

stock_info_df = pd.DataFrame.from_dict(discount_dict, orient='index').reset_index()
stock_info_df.columns = ['ticker','discount']
stock_info_df['sector'] = [pega_setor(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

,ticker,discount,sector
0,LVBI11.SA,-12.248686,Logística
1,KNHY11.SA,-4.262554,Títulos e Val. Mob.
2,JSRE11.SA,-18.457521,Híbrido
3,XPLG11.SA,-12.203314,Logística
4,HGLG11.SA,-5.016607,Logística
5,HGRE11.SA,-19.150068,Lajes Corporativas
6,TGAR11.SA,12.357693,Outros
7,BBPO11.SA,-23.589447,Lajes Corporativas
8,VRTA11.SA,-14.737031,Títulos e Val. Mob.
9,HFOF11.SA,-8.841878,Títulos e Val. Mob.


In [ ]:
def pega_dividendos(stock):
  return[x for x  in df[symbols == stock]['DividendYield']]

stock_info_df['DividendYield'] = [pega_dividendos(tick)[0] for tick in stock_info_df['ticker']] 
stock_info_df


,ticker,discount,sector,DividendYield
0,LVBI11.SA,-12.248686,Logística,0.50
1,KNHY11.SA,-4.262554,Títulos e Val. Mob.,1.05
2,JSRE11.SA,-18.457521,Híbrido,0.56
3,XPLG11.SA,-12.203314,Logística,0.52
4,HGLG11.SA,-5.016607,Logística,0.45
5,HGRE11.SA,-19.150068,Lajes Corporativas,0.49
6,TGAR11.SA,12.357693,Outros,0.90
7,BBPO11.SA,-23.589447,Lajes Corporativas,0.93
8,VRTA11.SA,-14.737031,Títulos e Val. Mob.,0.78
9,HFOF11.SA,-8.841878,Títulos e Val. Mob.,0.60


In [ ]:
std_list = []

for tick in stock_info_df['ticker']:
  std_list.append(returns[tick].std())

stock_info_df['risk'] = std_list 

stock_info_df.sort_values(by = 'discount').head(10)


KeyError: ignored

In [ ]:
def Quantidade_Ativos(stock):
  return[x for x  in df[symbols == stock]['QuantidadeAtivos']]

stock_info_df['QuantidadeAtivos'] = [Quantidade_Ativos(tick)[0] for tick in stock_info_df['ticker']]


In [ ]:
def Preco_atual(stock):
  return[x for x  in df[symbols == stock]['Preço Atual']]

stock_info_df['Preço Atual'] = [Preco_atual(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def DY12MM(stock):
  return[x for x  in df[symbols == stock]['DY12MM']]

stock_info_df['DY12MM'] = [DY12MM(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
#def Patrimonio_Liquido(stock):
#  return[x for x  in df[symbols == stock]['Patrimonio']]

#stock_info_df['Patrimonio.'] = [Patrimonio_Liquido(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def P_VPA(stock):
  return[x for x  in df[symbols == stock]['P/VPA']]

stock_info_df['P/VPA'] = [P_VPA(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def LiqDia(stock):
  return[x for x  in df[symbols == stock]['LiqDia']]

stock_info_df['LiqDia'] = [LiqDia(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def Dividendo(stock):
  return[x for x  in df[symbols == stock]['Dividendo']]

stock_info_df['Dividendo'] = [Dividendo(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
import plotly.express as px
stock_info_df = stock_info_df.dropna()

In [ ]:
fig = px.scatter(stock_info_df, x = 'risk', y ='discount', color = 'sector', hover_name='ticker', log_x = True)

plotly.offline.plot(fig,filename='oportunidades.html')

In [ ]:
stock_info_df

In [ ]:
#removendo caracteres

stock_info_df['Preço Atual'] = stock_info_df['Preço Atual'].str.replace(r'\D', '').astype(float)/100
#stock_info_df['DY (12M)Média'] = stock_info_df['DY (12M)Média'].str.replace(r'\D', '').astype(float)/100
#stock_info_df['DividendYield'] = stock_info_df['DividendYield'].str.replace(r'\D', '').astype(float)/100
#stock_info_df['PatrimônioLíq.'] = stock_info_df['PatrimônioLíq.'].str.replace(r'\D', '').astype(float)/100
#stock_info_df['PatrimônioLíq.'] = stock_info_df['PatrimônioLíq.'].str.replace(r'\D', '').astype(float)

In [ ]:
stock_info_df.dtypes

In [ ]:
# Filtros 
#stock_info_df =  stock_info_df.query('sector <> Residencial') 
#stock_info_df =  stock_info_df.query('DividendYield > 0.50')
#stock_info_df = [stock_info_df['sector'] >= 'Residencial']

In [ ]:
#type('PatrimônioLíq')
#stock_info_df = [stock_info_df['PatrimônioLíq.'] >= 'R$ 365.691.116,16']
#stock_info_df =  stock_info_df.query('PatrimônioLíq. >= 365.691.116,16')

In [ ]:
stock_info_df

In [ ]:
stock_info_df.to_excel('fiis.xlsx')